In [1]:
!pip install transformers datasets peft torch

In [3]:
import os
os.remove('/kaggle/working/fine-tuned-QA-tinyllama-1.1B')
os.remove('/kaggle/working/outputs')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/fine-tuned-QA-tinyllama-1.1B'

In [2]:
model_ref = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [10]:
# qa_data = [
#     {"question": "What was the revenue for AlphaTech Inc. in Q3?", "answer": "$80M"},
#     {"question": "How many new customers did AlphaTech acquire in Q2?", "answer": "25,000 new users"},
#     {"question": "Which quarter did AlphaTech enter the Asian market?", "answer": "Q3"},
#     {"question": "What was the net profit of AlphaTech in Q4?", "answer": "$15M"},
#     {"question": "What investment did AlphaTech make in Q1?", "answer": "Invested $5M in AI research."},
#     {"question": "What major challenge did AlphaTech face in Q3?", "answer": "Rising competition."},
#     {"question": "Which region did AlphaTech expand to in Q4?", "answer": "Latin America."},
#     {"question": "What was the main regulatory challenge faced by AlphaTech?", "answer": "Regulatory compliance challenges in Q4."}]

In [3]:
qa_data = [
    {"question": "What number becomes zero when you subtract 15 from half of it?", "answer": "30"},
    {"question": "What three positive numbers give the same answer when multiplied and added together?", "answer": "1, 2, and 3"},
    {"question": "I am a number. Divide me by 3, then add 5, and you get 11. What number am I?", "answer": "18"},
    {"question": "I am a number less than 50. If you double me and subtract 10, you get 30. What number am I?", "answer": "20"},
    {"question": "A farmer has 17 sheep, and all but 9 run away. How many are left?", "answer": "9"},
    {"question": "The sum of two numbers is 16, and their difference is 4. What are the numbers?", "answer": "10 and 6"},
    {"question": "Double a number and subtract 8 to get 10. What number is it?", "answer": "9"},
    {"question": "A clock shows 3:15. What is the angle between the hour and the minute hand?", "answer": "7.5 degrees"},
    {"question": "What number do you get if you divide 30 by half and add 10?", "answer": "70"},
    {"question": "Take me and double my value. Add 8, divide by 4, and you get 5. What number am I?", "answer": "6"},
    {"question": "A box contains 12 apples. You take away 5. How many do you have?", "answer": "5 (You took them!)"},
    {"question": "If five cats catch five mice in five minutes, how many cats are needed to catch 100 mice in 100 minutes?", "answer": "5 cats"},
    {"question": "The sum of three consecutive even numbers is 54. What are the numbers?", "answer": "16, 18, 20"},
    {"question": "I am a two-digit number. My tens digit is 3 more than my ones digit. What number am I?", "answer": "41"},
    {"question": "I am thinking of a number. When I add 10 to it and then divide by 2, I get 12. What number am I?", "answer": "14"}
]

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset
from transformers import TrainingArguments, Trainer
import gc


# Define the model name
model_name = model_ref

# Load pre-trained model & tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [5]:
def format_qa(example):
    return {
        "text": f"Question: {example['question']} Answer: {example['answer']}"
    }

qa_dataset = Dataset.from_list(qa_data)
formatted_dataset = qa_dataset.map(format_qa)

# Tokenization
def preprocess_function(examples):
    inputs = tokenizer(
        examples['text'],
        truncation=True,
        padding="max_length",
        max_length=512
    )

    # Labels should be a shifted version of input_ids for causal LM training
    inputs["labels"] = inputs["input_ids"].copy()
    return inputs

# Apply tokenization
tokenized_qa_dataset = formatted_dataset.map(preprocess_function, batched=True)


# Define LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)


# Set Training Arguments

training_args = TrainingArguments(
    per_device_train_batch_size=5,  # Adjusted for GPU memory limitations
    gradient_accumulation_steps=3,  # To simulate a larger batch size
    warmup_steps=100,
    max_steps=3,#Number of epochs
    learning_rate=2e-4,
    fp16=True,  # Enable mixed precision training
    logging_steps=10,
    output_dir="outputs",
    report_to="none",
    remove_unused_columns=False,
)

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [6]:
# Move model to CPU to free memory before training
model = model.to("cpu")

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_qa_dataset,
)

# Free up memory before training

gc.collect()  # Garbage collection
torch.cuda.empty_cache()  # Clears CUDA cache
print("GPU clache cleared")

# Optimize model with torch.compile (improves execution speed)
model = torch.compile(model)

# Move model back to GPU for training
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Start training
trainer.train()

# Save the fine tuned model
model.save_pretrained("fine-tuned-QA-tinyllama-1.1B")
tokenizer.save_pretrained("fine-tuned-QA-tinyllama-1.1B")

GPU clache cleared


Step,Training Loss


('fine-tuned-QA-tinyllama-1.1B/tokenizer_config.json',
 'fine-tuned-QA-tinyllama-1.1B/special_tokens_map.json',
 'fine-tuned-QA-tinyllama-1.1B/tokenizer.model',
 'fine-tuned-QA-tinyllama-1.1B/added_tokens.json',
 'fine-tuned-QA-tinyllama-1.1B/tokenizer.json')

In [7]:
# Load the fine-tuned model
model_path = "fine-tuned-QA-tinyllama-1.1B"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def generate_answer(question, max_length=50):
    prompt = f"Question: {question} Answer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length, temperature=0.7, top_k=50, top_p=0.9)

    return tokenizer.decode(output[0], skip_special_tokens=True)



In [9]:
# Example test cases - Known questions
test_questions = [
    "I am a two-digit number. My tens digit is 3 more than my ones digit. What number am I?"
]

for q in test_questions:
    print(f"Q: {q}")
    print(f"A: {generate_answer(q)}\n")


Q: I am a two-digit number. My tens digit is 3 more than my ones digit. What number am I?
A: Question: I am a two-digit number. My tens digit is 3 more than my ones digit. What number am I? Answer: The number is 123.

